# 1. Load mô hình và các thư viện cần dùng

In [ ]:
from collections import Counter
from keras.models import load_model
import nltk
import numpy as np
import re

model = load_model('/content/drive/MyDrive/spellcheck_model.h5')

NGRAM = 5
MAXLEN = 39

# Các ký tự có thể xuất hiện trong encoder
vowel = list(
    'aAăĂâÂáÁàÀảẢãÃạẠắẮằẰẳẲẵẴặẶấẤầẦẩẨẫẪậẬ'
    'eEêÊéÉèÈẻẺẽẼẹẸếẾềỀểỂễỄệỆ'
    'iIíÍìÌỉỈĩĨịỊ'
    'oOôÔơƠóÓòÒỏỎõÕọỌốỐồỒổỔỗỖộỘớỚờỜởỞỡỠợỢ'
    'uUưƯúÚùÙủỦũŨụỤứỨừỪửỬữỮựỰ'
    'yYýÝỳỲỷỶỹỸ'
)
full_letters = vowel + list('bBcCdDđĐgGhHkKlLmMnNpPqQrRsStTvVxXzZ')
alphabet = ['\x00', ' '] + list('0123456789') + full_letters

region = {
    # Lẫn lộn dấu hỏi và ngã
    'ả': 'ã', 'ã': 'ả',
    'Ả': 'Ã', 'Ã': 'Ả',
    'ẻ': 'ẽ', 'ẽ': 'ẻ',
    'Ẻ': 'Ẽ', 'Ẽ': 'Ẻ',
    'ỉ': 'ĩ', 'ĩ': 'ỉ',
    'Ỉ': 'Ĩ', 'Ĩ': 'Ỉ',
    'ỏ': 'õ', 'õ': 'ỏ',
    'Ỏ': 'Õ', 'Õ': 'Ỏ',
    'ủ': 'ũ', 'ũ': 'ủ',
    'Ủ': 'Ũ', 'Ũ': 'Ủ',
    'ỷ': 'ỹ', 'ỹ': 'ỷ',
    'Ỷ': 'Ỹ', 'Ỹ': 'Ỷ',

    # Một số vùng nói nhẹ dấu sắc thành ngang
    'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u', 'ý': 'y',
    'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U', 'Ý': 'Y',

    # Một số vùng lẫn sắc - hỏi hoặc sắc - ngã
    'á': 'ả', 'ả': 'á', 'á': 'ã', 'ã': 'á',
    'Á': 'Ả', 'Ả': 'Á', 'Á': 'Ã', 'Ã': 'Á',

    # Cặp phổ biến khác
    'ạ': 'a', 'ẹ': 'e', 'ị': 'i', 'ọ': 'o', 'ụ': 'u', 'ỵ': 'y',
    'Ạ': 'A', 'Ẹ': 'E', 'Ị': 'I', 'Ọ': 'O', 'Ụ': 'U', 'Ỵ': 'Y'
}


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 2. Hàm encoder và decoder tương ứng với mô hình đã huấn luyện

In [ ]:
# Đệm '\x00' vào cuối của các cụm ngrams có độ dài < độ dài tối đa là 39
# One-hot encoding
def _encoder_data(text):
    x = np.zeros((MAXLEN, len(alphabet)))
    i = 0
    for c in text[:MAXLEN]:
        if c in alphabet:  # Chỉ xử lý ký tự có trong alphabet
            x[i, alphabet.index(c)] = 1
            i += 1
        if i >= MAXLEN:
            break
    # Đệm '\x00' cho các vị trí còn lại
    for j in range(i, MAXLEN):
        x[j, 0] = 1
    return x

# Ghép các ký tự dựa vào vector one-hot
def _decoder_data(x):
    x = x.argmax(axis = -1)
    return ''.join(alphabet[i] for i in x)

In [ ]:
print(_encoder_data('Tôi tên là Nguyễn Văn An').shape)
print(_decoder_data(_encoder_data('Tôi tên là Nguyễn Văn An')))

(39, 190)
Tôi tên là Nguyễn Văn An               


# 3. Các hàm sẽ sử dụng để sửa lỗi chính tả trong văn bản Tiếng Việt

### Hàm tách các câu thành các ngrams

In [ ]:
def _nltk_ngrams(sentence, n, maxlen):
    list_ngrams = []
    list_words = sentence.split()
    num_words = len(list_words)

    if (num_words >= n):
        for ngram in nltk.ngrams(list_words, n):
            if len(' '.join(ngram)) <= maxlen:
                list_ngrams.append(ngram)
    else:
        list_ngrams.append(tuple(list_words))
    return list_ngrams

In [ ]:
_nltk_ngrams('Xuwr ný ngoon ngữ tuwj nhêin', NGRAM, MAXLEN)

[('Xuwr', 'ný', 'ngoon', 'ngữ', 'tuwj'),
 ('ný', 'ngoon', 'ngữ', 'tuwj', 'nhêin')]

### Hàm dự đoán ngram bằng mô hình

In [ ]:
def _guess(ngram):
    text = ' '.join(ngram)
    x = _encoder_data(text)                    # shape: (MAXLEN, len(alphabet))
    x = np.expand_dims(x, axis=0)              # shape: (1, MAXLEN, len(alphabet)) đúng chuẩn batch input
    preds = model.predict(x)                   # output shape: (1, MAXLEN, len(alphabet))
    return _decoder_data(preds[0]).strip('\x00')

In [ ]:
_guess(('Xuwr', 'ný', 'ngoon', 'ngữ', 'tuwj'))

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


'Xử lý ngồn ngg tu'

### Hàm thêm dấu câu sau khi sửa lỗi chính tả

In [ ]:
def _add_punctuation(text, corrected_text):
    list_punctuation = {}
    for (i,word) in enumerate(text.split()):
        if word[0] not in alphabet or word[-1] not in alphabet:
            #Dấu ở đầu chữ như " và '
            start_punc = ''
            for c in word:
                if c in alphabet:
                    break
                start_punc += c
            #Dấu ở sau chữ như .?!,;:
            end_punc = ''
            for c in word[::-1]:
                if c in alphabet:
                    break
                end_punc += c
            end_punc = end_punc[::-1]
            # Lưu vị trí từ và dấu câu trong từ đó
            list_punctuation[i] = [start_punc, end_punc]
    result = ''
    for (i, word) in enumerate(corrected_text.split()):
        if i in list_punctuation:
            result += (list_punctuation[i][0] + word + list_punctuation[i][1]) + ' '
        else:
            result += word + ' '
    return result.strip()

### Hàm sửa lỗi chính tả trong câu

In [ ]:
def _correct(text):
    #xoá các ký tự đặc biệt
    new_text = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)
    ngrams = list(_nltk_ngrams(new_text, NGRAM, MAXLEN))
    guessed_ngrams = list(_guess(ngram) for ngram in ngrams)
    candidates = [Counter() for _ in range(len(guessed_ngrams) + NGRAM - 1)]
    for nid, ngram in (enumerate(guessed_ngrams)):
        for wid, word in (enumerate(re.split('\s', ngram))):
            candidates[nid + wid].update([word])
    corrected_text = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
    return _add_punctuation(text, corrected_text)

In [ ]:
text = 'Xuwr ný ngoon ngữ tuwj nhêin'
_correct(text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step


'Xử lý ngồn ngg tuwj nhiên'

# 4. Kết quả khi áp dụng mô hình

In [ ]:
#Nhập vào văn bản muốn sửa lỗi chính tả
text = input()

#Văn bản sau khi sửa lỗi chính tả
result = _correct(text)
print(result)

#Xoá bỏ ký tự đặc biệt
text = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)
list_text = text.split()

result = re.sub(r'[^' + ''.join(alphabet) + ']', '', result)
list_result = result.split()

#Hiển thị những từ đã sửa
corrected_word = [(list_text[i], list_result[i]) for i in range(len(list_text)) if list_text[i] != list_result[i]]
corrected_word

hômm nay laf thuws nam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
hômm nay làf thứ nam


[('la', 'là'), ('thus', 'thứ')]